In [ ]:
# This script pre processes the data and runs the decoding needed for the window analysis 

In [1]:
#imports
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import seaborn as sns
#from dataClasses import Trial,Neuron
from sklearn.feature_selection import RFECV
from sklearn import svm
from sklearn.model_selection import cross_validate, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report
#import intertools 
# Import Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from imblearn.pipeline import  Pipeline

from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import boto3
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy import stats
import numpy as np
from sklearn.metrics import confusion_matrix
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline

In [2]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Defining the RNN layer
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        
        # RNN forward pass
        out, _ = self.rnn(x)
        
        # Use only the last time step's output
        out = out[:, -1]  # Take the output from the last sequence step
        
        # Fully connected layer
        out = self.fc(out)
        
        return out


In [3]:
def get_data_ready(brain_area, type_of_analysis):
    all_trials=np.load("processed_data/"+ brain_area + "_all_trials.npy",allow_pickle=True)
    trial_list=np.empty((all_trials.shape[0],3))
    trial_dff=np.empty((all_trials.shape[0],41)) 
    trial_licks=np.empty((all_trials.shape[0],41))
    # Here we get the data based on the conditions that we want (i.e: if it is is lick, or go) 
    
    for trial in range(all_trials.shape[0]):
        
        trial_list[trial,0]=all_trials[trial].neuron_num
    
        
        if type_of_analysis=="GO":
            # for labelling go nogo
            if (all_trials[trial].trial_type)=="go":
                trial_list[trial,1]=1
            elif (all_trials[trial].trial_type)=="nogo":
                trial_list[trial,1]=0
            else:
                print("gone wrong on neuron ",all_trials[trial].neuron_num )
        elif type_of_analysis=="LICK":    
            #for labelling lick nolick
            if (all_trials[trial].trial_outcome)=="FA":
                trial_list[trial,1]=1
            elif (all_trials[trial].trial_outcome)=="Hit":
                trial_list[trial,1]=1
            elif (all_trials[trial].trial_outcome)=="Miss":
                trial_list[trial,1]=0
            elif (all_trials[trial].trial_outcome)=="CR":
                trial_list[trial,1]=0
            else:
                print("gone wrong on neuron ",all_trials[trial].neuron_num )
        elif type_of_analysis=="FA/HIT":
             if (all_trials[trial].trial_outcome)=="FA":
                 trial_list[trial,1]=1
             elif (all_trials[trial].trial_outcome)=="Hit":
                 trial_list[trial,1]=0
             elif (all_trials[trial].trial_outcome)=="Miss":
                 trial_list[trial,1]=2
             elif (all_trials[trial].trial_outcome)=="CR":
                 trial_list[trial,1]=2
             #filter to have just the 0 and 1 trials later
        else:
            print("Type of analysis can only be 'GO' or 'LICK'")
        
        trial_list[trial,2]=all_trials[trial].mouse_id
        if (all_trials[trial].trial_outcome)=="FA" or (all_trials[trial].trial_outcome)=="Hit":
            trial_dff[trial,0:41]=all_trials[trial].dff 
            #rebase  dff 
            trial_licks[trial,:]=all_trials[trial].licks
            lick_start=np.argmax(trial_licks[trial,:]==1,axis=0)
            trial_dff_rel_licks=np.zeros(trial_licks.shape[1])
            new_start=lick_start-4
            if new_start>=0:                    
                # move the array of licks left so that timeseries starst at lick time
                trial_dff_rel_licks[0:trial_licks.shape[1]-new_start]=all_trials[trial].dff[new_start: ]
                ## need to mske the last few timesteps on the new series = final timestep on original series
                trial_dff_rel_licks[trial_licks.shape[1]-new_start :]=all_trials[trial].dff[-1]
            else: 
                # move the array of licks right so that timeseries starst at lick time
                trial_dff_rel_licks[0-new_start :]=all_trials[trial].dff[0:all_trials[trial].dff.shape[0]+new_start]
                ## need to mske the first few timesteps on the new series = first timestep on original series
                trial_dff_rel_licks[0:-new_start]=all_trials[trial].dff[ 0 ]
                #trial_dff[trial,41:82]=trial_dff_rel_licks
        else:
            trial_dff[trial,0:41]=all_trials[trial].dff 
            #trial_dff[trial,41:82]=all_trials[trial].dff
    return trial_list, trial_dff


In [4]:
import numpy as np
import pandas as pd

def process_trials_for_group(trial_list, trial_dff):
    """
    Groups neuronal data for each trial and assigns a session ID.

    Args:
        trial_list (list): List of trial objects with attributes (trial_id, mouse_id, date, num_trial).
        trial_dff (list): List of neuronal data arrays (one per neuron, across trials).

    Returns:
        pd.DataFrame: DataFrame grouping neuronal data per trial and assigning a session ID.
        dict: Dictionary mapping session IDs to trial IDs.
    """
    # ✅ Define grouping variables
    variables_of_interest = ['mouse_id', 'date', 'num_trial', 'trial_id']

    # ✅ Extract relevant trial metadata
    trials_data = [{key: getattr(trial, key) for key in variables_of_interest} for trial in trial_list]
    trial_df = pd.DataFrame(trials_data)

    # ✅ Group all neurons belonging to the same trial
    grouped_trials = trial_df.groupby(['mouse_id', 'date', 'num_trial', 'trial_id'])

    # ✅ Store grouped neuronal data and trial IDs
    session_data = []
    session_trial_ids = {}

    for trial_key, indices in grouped_trials.groups.items():
        mouse_id, date, num_trial, trial_id = trial_key
        session_id = f"{mouse_id}_{date}"  # Unique session identifier

        # Stack neuronal data for this trial (Neurons, Timepoints)
        trial_dff_grouped = np.vstack([trial_dff[idx] for idx in indices])

        # Append to session data
        session_data.append({
            "session_id": session_id,
            "num_trial": num_trial,
            "trial_id": trial_id,  # This is the y value (classification label)
            "dff": trial_dff_grouped  # (Neurons, Timepoints)
        })

        # Store trial IDs for each session
        if session_id not in session_trial_ids:
            session_trial_ids[session_id] = []
        session_trial_ids[session_id].append(trial_id)

    # ✅ Convert to DataFrame
    session_df = pd.DataFrame(session_data)

    return session_df


In [94]:
def process_trials_for_group(trial_list, trial_dff):
    """
    Groups all neurons from the same trial into one dataset.

    Args:
        trial_list (list): List of trial objects with attributes (e.g., trial_id, mouse_id, date, num_trial).
        trial_dff (list): List of neuronal data arrays corresponding to each trial.

    Returns:
        pd.DataFrame: DataFrame grouping dff of all neurons for each session.
        list: List mapping session identifiers to trial_ids.
    """
    # Variables of interest for grouping
    variables_of_interest = ['trial_id', 'mouse_id', 'date']#, 'num_trial']

    # Extract relevant info from trial_list
    trials_data = [{key: getattr(trial, key) for key in variables_of_interest} for trial in trial_list]
    trial_df = pd.DataFrame(trials_data)

    # Group by unique trial identifiers: mouse_id, date, num_trial
    grouped_trials = trial_df.groupby(['mouse_id', 'date'])#, 'num_trial'])

    session_trial_ids = []
    session_data = []

    for session_key, group_indices in grouped_trials.groups.items():
        # Retrieve neuronal data for all trials in this group
        session_dff_list = [trial_dff[idx] for idx in group_indices]

        # Stack ALL neurons together
        session_dff = np.vstack(session_dff_list)  # (Total Neurons, Timepoints)

        # Store session data
        session_data.append({
            "session": session_key,  # Full session key
            "dff": session_dff  # All neurons concatenated
        })

        # Store trial IDs
        session_trial_ids.append( trial_df.iloc[list(group_indices)].trial_id.tolist())

    # Create DataFrame for session-wise grouped data
    session_data = pd.DataFrame(session_data)

    return session_data, session_trial_ids


In [8]:
#First we rearrange the data to have all the neurons for one session in one dataframe (num_trials, num_time_steps, num_neurons)
def get_pop_data_ready(brain_area, type_of_analysis):
    all_trials=np.load("processed_data/"+brain_area + "_all_trials.npy",allow_pickle=True)
    trial_list=np.empty((all_trials.shape[0],4))
    trial_dff=np.empty((all_trials.shape[0],41)) 
    trial_licks=np.empty((all_trials.shape[0],41))
    
    for trial in range(all_trials.shape[0]):
        
        if type_of_analysis=="GO":
            # for labelling go nogo
            if (all_trials[trial].trial_type)=="go":
                all_trials[trial].trial_id=1
            elif (all_trials[trial].trial_type)=="nogo":
                all_trials[trial].trial_id=0
            else:
                print("gone wrong on neuron ",all_trials[trial].neuron_num )
        elif type_of_analysis=="LICK":
            #for labelling lick nolick
            if (all_trials[trial].trial_outcome)=="FA":
                all_trials[trial].trial_id=1
            elif (all_trials[trial].trial_outcome)=="Hit":
                all_trials[trial].trial_id=1
            elif (all_trials[trial].trial_outcome)=="Miss":
                all_trials[trial].trial_id=0
            elif (all_trials[trial].trial_outcome)=="CR":
                all_trials[trial].trial_id=0
            else:
                print("gone wrong on neuron ",all_trials[trial].neuron_num)
        elif type_of_analysis=="FA/HIT":
            if (all_trials[trial].trial_outcome)=="FA":
                all_trials[trial].trial_id=1
            elif (all_trials[trial].trial_outcome)=="Hit":
                all_trials[trial].trial_id=0
            elif (all_trials[trial].trial_outcome)=="Miss":
                all_trials[trial].trial_id=2
            elif (all_trials[trial].trial_outcome)=="CR":
                all_trials[trial].trial_id=2
        else:
            print("Type of analysis can only be 'GO' or 'LICK'")
    
        if (all_trials[trial].trial_outcome)=="FA" or (all_trials[trial].trial_outcome)=="Hit":
            trial_dff[trial,0:41]=all_trials[trial].dff 
            #rebase  dff 
            trial_licks[trial,:]=all_trials[trial].licks
            lick_start=np.argmax(trial_licks[trial,:]==1,axis=0)
            trial_dff_rel_licks=np.zeros(trial_licks.shape[1])
            new_start=lick_start-4
            if new_start>=0:                    
                # move the array of licks left so that timeseries starst at lick time
                trial_dff_rel_licks[0:trial_licks.shape[1]-new_start]=all_trials[trial].dff[new_start: ]
                ## need to mske the last few timesteps on the new series = final timestep on original series
                trial_dff_rel_licks[trial_licks.shape[1]-new_start :]=all_trials[trial].dff[-1]
            else: 
                # move the array of licks right so that timeseries starst at lick time
                trial_dff_rel_licks[0-new_start :]=all_trials[trial].dff[0:all_trials[trial].dff.shape[0]+new_start]
                ## need to mske the first few timesteps on the new series = first timestep on original series
                trial_dff_rel_licks[0:-new_start]=all_trials[trial].dff[ 0 ]
                #trial_dff[trial,41:82]=trial_dff_rel_licks
        else:
            trial_dff[trial,0:41]=all_trials[trial].dff 
            #trial_dff[trial,41:82]=all_trials[trial].dff
    return all_trials, trial_dff

In [ ]:
# Run the analysis for different windows for each neuron
num_permutations = 1000
results_window = []

# Brain areas and analysis types
brain_area = ("N")
type_of_analysis = ("FA/HIT")

for brain in brain_area:
    trial_list, trial_dff = get_data_ready("S1naive", type_of_analysis)
    neuron_list = np.unique(trial_list[:, 0]).reshape(-1, 1)

    print("Num of neurons before filter=", neuron_list.shape[0])

    size_of_window = [8]

    for size in size_of_window:
        window_start = np.arange(0, 45 - size)
        for start in window_start:
            for neuron in neuron_list:
                print("\n", neuron, "XXXXXXXXXXXXXXXXXXXX")

                # Extract neuron-specific data
                neuron_data_list = trial_list[trial_list[:, 0] == neuron].copy()
                neuron_data_dff = trial_dff[trial_list[:, 0] == neuron].copy()

                ### select only FA/MISS trials by only getting the ones that have 0 or 1
                valid_indices = np.where((neuron_data_list == 0) | (neuron_data_list == 1))[0]
                print(f"Valid indices: {valid_indices}")
                print(f"Number of valid indices: {len(valid_indices)}")

                if len(valid_indices) < 10:
                    print(f"Skipping neuron {neuron} due to insufficient valid indices.")
                    continue
                # Use these indices to extract the corresponding elements from neuron_data_dff
                filtered_dff = neuron_data_dff[valid_indices]
                filtered_list= neuron_data_list[valid_indices]
                # Get window-specific data
                neuron_data_dff = filtered_dff[:, start : start + size]

                X = neuron_data_dff.copy()
                y = filtered_list[:, 1]

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.2, random_state=42
                )

                # Reshape for RNN input: Add an extra dimension
                X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
                X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

                # Convert to PyTorch tensors
                Xtrain = torch.from_numpy(X_train).float()
                ytrain = torch.from_numpy(y_train).long()
                Xtest = torch.from_numpy(X_test).float()
                ytest = torch.from_numpy(y_test).long()
                print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
                print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
               
                # Batch size, epochs, and iterations
                batch_size = 16
                n_iters = 1000
                num_epochs = int(n_iters // (len(X_train) /batch_size))

                train = TensorDataset(Xtrain, ytrain)
                test = TensorDataset(Xtest, ytest)

                train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=False)
                test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=False)
              
                # Create the RNN model
                input_dim = X_train.shape[-1]
                hidden_dim = 4
                layer_dim = 1
                output_dim = 1

                model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

                # Loss function and optimizer
                error = nn.BCEWithLogitsLoss()  # No class weights, since oversampling handles imbalance
                learning_rate = 0.01
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

                # Training loop for the original model
                train_accuracy_list = []
                test_accuracy_list = []
                test_loss_list = []

                for epoch in range(num_epochs):
                    model.train()
                    total_train, correct_train = 0, 0
                
                    for X_batch, y_batch in train_loader:
                        # Skip empty batches
                        if y_batch.size(0) == 0:
                            print("Skipping empty batch.")
                            continue
                
                        optimizer.zero_grad()
                        outputs = model(X_batch).squeeze()
                        loss = error(outputs, y_batch.float())
                        loss.backward()
                        optimizer.step()
                
                        # Calculate training accuracy
                        probabilities = torch.sigmoid(outputs)
                        predicted = (probabilities > 0.5).long()
                        correct_train += (predicted.squeeze() == y_batch).sum().item()
                        total_train += y_batch.size(0)
                
                    if total_train == 0:
                        print("No training samples processed. Skipping accuracy calculation for this epoch.")
                        continue
                
                    train_accuracy = 100 * correct_train / total_train
                    train_accuracy_list.append(train_accuracy)
                    scheduler.step()
                    # Evaluate on test set
                    model.eval()
                    total_test, correct_test = 0, 0

                    with torch.no_grad():
                        for X_test_batch, y_test_batch in test_loader:
                            outputs = model(X_test_batch).squeeze(dim=-1)
                            probabilities = torch.sigmoid(outputs)
                            predicted = (probabilities > 0.5).long()
                            correct_test += (predicted == y_test_batch).sum().item()
                            total_test += y_test_batch.size(0)
                            loss_test = error(outputs, y_test_batch.float())
                            test_loss_list.append(loss_test)

                    test_accuracy = 100 * correct_test / float(total_test)
                    test_accuracy_list.append(test_accuracy)

                # Save the original model's test accuracy
                original_test_accuracy = test_accuracy

                # Random classifier: Match class distribution in the test set
                class_distribution = np.mean(y_test)  # proportion of class 1

                random_accuracies = []
                for _ in range(num_permutations):
                    y_random = np.random.choice(
                        [0, 1], size=len(y_test), p=[1 - class_distribution, class_distribution]
                    )
                    random_accuracy = accuracy_score(y_test, y_random)
                    random_accuracies.append(random_accuracy)

                # Calculate p-value using a t-test (testing original model vs. random classifiers)
                random_accuracies = np.array(random_accuracies)
                p_value = np.mean(random_accuracies >= original_test_accuracy / 100)

                # Store results
                results_window.append(
                    {
                        "Brain Area": brain,
                        "Window Size": size,
                        "Window Start": start,
                        "Neuron": neuron,
                        "Train Accuracy": train_accuracy_list,
                        "Test Accuracy": original_test_accuracy,
                        "p value": p_value,
                    }
                )

                print(f"Neuron {neuron}: Test Accuracy = {original_test_accuracy:.2f}%, p-value = {p_value:.4f}")

                # Save results to CSV
                results_df = pd.DataFrame(results_window)
                results_df.to_csv("running_results_S1naive_FAHIT.csv", index=False)


Num of neurons before filter= 318

 [1.21820191e+12] XXXXXXXXXXXXXXXXXXXX
Valid indices: [  8  51 176]
Number of valid indices: 3
Skipping neuron [1.21820191e+12] due to insufficient valid indices.

 [1.21820191e+12] XXXXXXXXXXXXXXXXXXXX
Valid indices: [  3   5   6   7   8  11  12  14  15  16  17  18  23  24  25  26  27  28
  29  30  31  32  33  34  36  37  39  40  41  43  44  46  47  49  53  54
  56  57  58  60  61  62  64  65  66  67  68  69  71  72  73  74  75  76
  78  84  85  87 107 108 109 111 112 113 114 117 120 121 123 124 128 129
 130 140 143 145 146 147 152 153 155 156 157 159 171 173 174 175 176 178
 179 183 193 194 195 196 204 215 219 220 223 225 227 228 233 243 250 251
 279 281 297 298 304 308 314 316]
Number of valid indices: 116
X_train shape: (92, 8, 1), y_train shape: (92,)
X_test shape: (24, 8, 1), y_test shape: (24,)
Neuron [1.21820191e+12]: Test Accuracy = 54.17%, p-value = 0.4240

 [1.21820191e+12] XXXXXXXXXXXXXXXXXXXX
Valid indices: [  3   4   6   7   8   9  10  1

In [96]:
def process_trials_for_group(trial_list, trial_dff):
    """
    Groups all neurons from the same session while preserving trials.

    Args:
        trial_list (list): List of trial objects with attributes (e.g., trial_id, mouse_id, date).
        trial_dff (list): List of neuronal data arrays corresponding to each trial.

    Returns:
        pd.DataFrame: DataFrame where dff shape is (num_neurons, num_trials, timepoints).
        list: List of trial_ids per session, shape (num_trials,).
    """
    # Variables of interest for grouping
    variables_of_interest = ['trial_id', 'mouse_id', 'date']

    # Extract relevant info from trial_list
    trials_data = [{key: getattr(trial, key) for key in variables_of_interest} for trial in trial_list]
    trial_df = pd.DataFrame(trials_data)

    # Group by unique session identifiers (mouse_id, date)
    grouped_trials = trial_df.groupby(['mouse_id', 'date'])

    session_trial_ids = []
    session_data = []

    for session_key, group_indices in grouped_trials.groups.items():
        # Retrieve neuronal data for all trials in this session
        session_dff_list = [trial_dff[idx] for idx in group_indices]  # List of (neurons, timepoints)

        # Convert list of trials into a 3D array (num_neurons, num_trials, timepoints)
        session_dff = np.stack(session_dff_list, axis=1)  # (num_neurons, num_trials, timepoints)

        # Store session data
        session_data.append({
            "session": session_key,  # Full session key
            "dff": session_dff  # All neurons stacked per trial
        })

        # Store trial IDs
        session_trial_ids.append(np.array(trial_df.iloc[list(group_indices)].trial_id.tolist()))

    # Convert session data to DataFrame
    session_data = pd.DataFrame(session_data)

    return session_data, session_trial_ids


In [ ]:
# now get the data for the population 

# Run the analysis for different windows for each neuron
num_permutations = 1000
results_pop_window = []

# Brain areas and analysis types
brain_area = ("S")
type_of_analysis = ("FA/HIT")
combinations = [('S1naive', "FA/HIT"),('PPC', "FA/HIT"),('S1', "FA/HIT")]
size_of_window = [8]


for brain_area, analysis_type in combinations:
    print(f"Brain Area: {brain_area}, Analysis Type: {analysis_type}")

    #load data
    trial_list, trial_dff = get_pop_data_ready(brain_area, analysis_type)
   
    session_result_df = process_trials_for_group(trial_list, trial_dff)
    sessions = np.unique(session_result_df['session_id'])
    print(f"There are {len(sessions)} sessions in {brain_area}")
    for session in sessions: 
        print(f"working out session {session} in {brain_area}")
        mask = session_result_df['session_id'] == session
        all_session_X = session_result_df[mask]
        all_X = np.array([np.array(trial) for trial in all_session_X['dff']])  # Ensure consistent shape
        all_y = np.array([np.array(trial) for trial in all_session_X['trial_id']])  # Ensure correct indices
        valid_indices = np.where((all_y == 0) | (all_y == 1))[0]
       # print(f"Valid indices: {valid_indices}")
        print(f"Number of valid indices: {len(valid_indices)}")
        
        if len(valid_indices) < 10:
            print(f"Skipping session {session} due to insufficient valid indices.")
            continue

                  
        # Use these indices to extract the corresponding elements from neuron_data_dff
        filtered_dff = all_X[valid_indices]
        print(f"filtered_dff shape: {filtered_dff.shape} (trial, neuron, timepoints)")

        filtered_list= all_y[valid_indices]
               

        for size in size_of_window:
            window_start = np.arange(0, 45 - size)
            for start in window_start:
                  
                # Get window-specific data
                window_dff = filtered_dff[:, :,start : start + size]
                #window_list= filtered_list[:, start : start + size]

                X = window_dff.copy()
                y = filtered_list.copy()

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.2, random_state=42)


                sequence_length = X_train.shape[1]

                # Reshape for RNN input: Add an extra dimension
                #X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])  
                #print(f"X_train reshaped: {X_train.shape}")

                #X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

                # Convert to PyTorch tensors
                Xtrain = torch.from_numpy(X_train).float()
                ytrain = torch.from_numpy(y_train).long()
                Xtest = torch.from_numpy(X_test).float()
                ytest = torch.from_numpy(y_test).long()
                print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
                print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
               
                # Batch size, epochs, and iterations
                batch_size = 16
                n_iters = 1000
                num_epochs = int(n_iters // (len(X_train) /batch_size))

                train = TensorDataset(Xtrain, ytrain)
                test = TensorDataset(Xtest, ytest)

                train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=False)
                test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=False)
              
                # Create the RNN model
                input_dim = X_train.shape[-1]
                hidden_dim = 4
                layer_dim = 1
                output_dim = 1

                model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

                # Loss function and optimizer
                error = nn.BCEWithLogitsLoss()  # No class weights, since oversampling handles imbalance
                learning_rate = 0.01
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

                # Training loop for the original model
                train_accuracy_list = []
                test_accuracy_list = []
                test_loss_list = []

                for epoch in range(num_epochs):
                    model.train()
                    total_train, correct_train = 0, 0
                
                    for X_batch, y_batch in train_loader:
                        # Skip empty batches
                        if y_batch.size(0) == 0:
                            print("Skipping empty batch.")
                            continue
                
                        optimizer.zero_grad()
                        outputs = model(X_batch).squeeze()
                        loss = error(outputs, y_batch.float())
                        loss.backward()
                        optimizer.step()
                
                        # Calculate training accuracy
                        probabilities = torch.sigmoid(outputs)
                        predicted = (probabilities > 0.5).long()
                        correct_train += (predicted.squeeze() == y_batch).sum().item()
                        total_train += y_batch.size(0)
                
                    if total_train == 0:
                        print("No training samples processed. Skipping accuracy calculation for this epoch.")
                        continue
                
                    train_accuracy = 100 * correct_train / total_train
                    train_accuracy_list.append(train_accuracy)
                    scheduler.step()
                    # Evaluate on test set
                    model.eval()
                    total_test, correct_test = 0, 0

                    with torch.no_grad():
                        for X_test_batch, y_test_batch in test_loader:
                            outputs = model(X_test_batch).squeeze(dim=-1)
                            probabilities = torch.sigmoid(outputs)
                            predicted = (probabilities > 0.5).long()
                            correct_test += (predicted == y_test_batch).sum().item()
                            total_test += y_test_batch.size(0)
                            loss_test = error(outputs, y_test_batch.float())
                            test_loss_list.append(loss_test)

                    test_accuracy = 100 * correct_test / float(total_test)
                    test_accuracy_list.append(test_accuracy)

                # Save the original model's test accuracy
                original_test_accuracy = test_accuracy

           
                # Store results
                results_pop_window.append(
                    {
                        "Brain Area": brain_area,
                        "Window Size": size,
                        "Window Start": start,
                        "Session": session,
                        "Train Accuracy": train_accuracy_list,
                        "Test Accuracy": original_test_accuracy
                    }
                )

                print(f"Session {session}: Test Accuracy = {original_test_accuracy:.2f}%")

                # Save results to CSV
                results_df = pd.DataFrame(results_pop_window)
                results_df.to_csv("running_results_pop_s1naive.csv", index=False)





Brain Area: S1naive, Analysis Type: FA/HIT
There are 14 sessions in S1naive
working out session 1218_20190701 in S1naive
Number of valid indices: 3
Skipping session 1218_20190701 due to insufficient valid indices.
working out session 1218_20190705 in S1naive
Number of valid indices: 0
Skipping session 1218_20190705 due to insufficient valid indices.
working out session 1218_20190712 in S1naive
Number of valid indices: 116
filtered_dff shape: (116, 4, 41) (trial, neuron, timepoints)
X_train shape: (92, 4, 8), y_train shape: (92,)
X_test shape: (24, 4, 8), y_test shape: (24,)
Session 1218_20190712: Test Accuracy = 66.67%
X_train shape: (92, 4, 8), y_train shape: (92,)
X_test shape: (24, 4, 8), y_test shape: (24,)
Session 1218_20190712: Test Accuracy = 58.33%
X_train shape: (92, 4, 8), y_train shape: (92,)
X_test shape: (24, 4, 8), y_test shape: (24,)
Session 1218_20190712: Test Accuracy = 66.67%
X_train shape: (92, 4, 8), y_train shape: (92,)
X_test shape: (24, 4, 8), y_test shape: (24,)

ValueError: Target size (torch.Size([1])) must be the same as input size (torch.Size([]))

In [23]:
y.shape

(442,)

In [78]:
print(len(sessions), all_session_X['dff'].shape, filtered_dff.shape)


27 (231,) (185, 15, 41)


In [77]:
np.array(all_session_X['dff'][7]).shape

(15, 41)

In [ ]:
# Convert results_window to a DataFrame
results_df = pd.DataFrame(results_window)

# Display the first few rows
print(results_df.head())

# Check for missing or unexpected values   
print(results_df.info())
print(results_df.describe())



# Check unique values for grouping keys
print("Unique brain areas:", results_df['Brain Area'].unique())
print("Unique window sizes:", results_df['Window Size'].unique())
print("Unique window starts:", results_df['Window Start'].unique())